In [ ]:
# Acá deberían cargar todas las librerías que vayan a usar. 
# Es preferible cargar las librerías al principio para que quede claro que requerimientos tiene la notebook. 
# Tengan cuidado de no cargar librerías que no vayan a utilizar ya que esto incurre en un gasto de memoria innecesario

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [ ]:
#Podriamos cargar los datos con la API de Kaggle sin necesidad de bajarlos, o alternativamente subirlos y no correr esta celda

import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c examen-2-ap-itba


import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

Mounted at /content/drive
 68% 9.00M/13.3M [00:00<00:00, 8.65MB/s]
100% 13.3M/13.3M [00:00<00:00, 27.2MB/s]
 85% 33.0M/38.6M [00:01<00:00, 15.4MB/s]
100% 38.6M/38.6M [00:01<00:00, 25.1MB/s]


In [ ]:
#Levantamos los datos (si los tenemos local y no corrimos la celda anterior, habría que subirlos)

df_train=pd.read_csv("datos.csv", sep=",")
df_test=pd.read_csv("resultados.csv", sep=",")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Después deberían hacer un poco de EDA para conocer la base
df_train.columns

Index(['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_x', 'directors', 'writers',
       'seasonNumber', 'episodeNumber', 'ordering', 'language', 'types',
       'attributes', 'isOriginalTitle', 'adult', 'budget', 'genres_y',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'video', 'id'],
      dtype='object')

In [ ]:
df_train.shape

(900000, 32)

In [ ]:
#Clasifico en categóricas y continuas
cat_vars = ['language'] 
contin_vars = ['startYear', 'endYear', 'numVotes', 'runtimeMinutes', 'budget', 'revenue']

In [ ]:
#Elimino los NA de las variables continuas
for v in contin_vars: df_train.loc[df_train[v].isnull(), v] = 0
for v in contin_vars: df_train.loc[(df_train[v].isna()), v] = 0

In [ ]:
# Genero variables dummys con las categóricas
for v in cat_vars: df_train=df_train.join(pd.get_dummies(df_train[v], drop_first=True), lsuffix = '_left', rsuffix = '_right')

In [ ]:
#Selección de variables para el modelo
algunas=contin_vars+['en', 'es']

In [ ]:
#Elimino los NA de las variables continuas y algún otro error que pueda aparecer

df_train=df_train.replace('\\N',0)
df_train=df_train.replace('Game-Show',0)

for v in algunas: df_train.loc[df_train[v].isnull(), v] = 0
for v in algunas: df_train.loc[(df_train[v].isna()), v] = 0

In [52]:
# Elegir un modelo y correrlo
# Recuerden que lo que tienen que clasificar es la columna averageRating

reg=RandomForestRegressor(max_depth=20)

X=np.array(df_train[algunas])
y=df_train['averageRating']

res=reg.fit(X, y)

In [53]:
res.score(X,y)

0.33390462113484315

In [ ]:
# Luego podría corresponder ajustar hiperparámetros, hacer CV, etc...

In [ ]:
#Elimino los NA de las variables continuas
for v in contin_vars: df_test.loc[df_test[v].isnull(), v] = 0
for v in contin_vars: df_test.loc[(df_test[v].isna()), v] = 0

In [ ]:
# Genero variables dummys con las categóricas
for v in cat_vars: df_test=df_test.join(pd.get_dummies(df_test[v], drop_first=True), lsuffix = '_left', rsuffix = '_right')

In [ ]:
df_test=df_test.replace('\\N',0)
df_test=df_test.replace('Reality-TV',0)

In [58]:
# Y hago una predicción con las variables de testeo
X=np.array(df_test[algunas])
resultado=reg.predict(X)

In [62]:
# Finalmente generar un CSV con los resultados aplicados al CSV de resultados que no contiene la columna target

salida = pd.DataFrame(data={"averageRating": resultado}).astype(str)
salida.index = df_test.id_left
salida.to_csv("RF.csv", sep=',',index=True,  index_label='id')

In [55]:
resul=pd.read_csv("derived.csv", sep=",")

In [56]:
res.score(X,resul.averageRating)

0.22736365830518146